# Base BERT implementation

1. Import dependencies

In [1]:
import random
import time

import numpy as np
import torch
import os

from transformers import (
    AutoConfig,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
)

from src.util.torch_device import resolve_torch_device
from src.data.span_detection_ds import ManipulationDetectionDataset
from src.definitions import (
    MODELS_FOLDER,
    RAW_DATA_FOLDER,
    PROCESSED_DATA_FOLDER,
)
from src.model.span_detection_metrics import compute_metrics

2. Prepare Env

In [2]:
random_seed = 42

random.seed(random_seed)
torch.manual_seed(random_seed)
np.random.seed(random_seed)

device = resolve_torch_device()

model_checkpoint = "distilbert/distilbert-base-multilingual-cased"

epoch_time = int(time.time())

os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"

3. Load dataset

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

dataset_blueprint = ManipulationDetectionDataset(
    tokenizer=tokenizer,
    raw_path=RAW_DATA_FOLDER / "span-detection.parquet",
    processed_path=PROCESSED_DATA_FOLDER / "span-detection",
    seed=random_seed,
)

dataset = dataset_blueprint.read()

3. Prepare model

In [4]:
training_args = TrainingArguments(
    output_dir=MODELS_FOLDER / "manipulation-detector-bert-ner-checkpoint",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    num_train_epochs=20,
    weight_decay=0.8,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    seed=random_seed,
    logging_steps=200,
    auto_find_batch_size=True,
    torch_empty_cache_steps=1000,
)

config = AutoConfig.from_pretrained(
    model_checkpoint,
    num_labels=len(dataset_blueprint.label2id),
    id2label=dataset_blueprint.id2label,
    label2id=dataset_blueprint.label2id,
    dropout=0.6,
)


def optuna_hp_space(trial):
    return {}


def model_init(trial):
    if trial:
        config.update(
            {
                "dropout": trial.suggest_float("dropout", 0.1, 0.8, log=True),
            }
        )
        training_args.learning_rate = trial.suggest_float(
            "learning_rate", 1e-6, 1e-4, log=True
        )
        training_args.weight_decay = trial.suggest_float(
            "weight_decay", 0.01, 0.9, log=True
        )

    return AutoModelForTokenClassification.from_pretrained(
        model_checkpoint,
        config=config,
    )


data_collator = DataCollatorForTokenClassification(tokenizer)


trainer = Trainer(
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics(dataset_blueprint),
    model_init=model_init,
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


4. Run HP search

In [5]:
def compute_objective(metrics):
    return metrics["eval_f1"]


best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=optuna_hp_space,
    n_trials=30,
    compute_objective=compute_objective,
)

[I 2025-02-06 23:50:40,349] A new study created in memory with name: no-name-63e3659c-47c3-4d85-bce4-76c69c303672
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.530000,0.495145,0.012308,0.010165,0.011134,0.749054


[I 2025-02-06 23:52:55,821] Trial 0 finished with value: 0.011134307585247043 and parameters: {'dropout': 0.2153605042391882, 'learning_rate': 3.668086206434241e-06, 'weight_decay': 0.1757935022534685}. Best is trial 0 with value: 0.011134307585247043.
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.490200,0.458914,0.035126,0.040661,0.037691,0.763192


[I 2025-02-06 23:55:06,627] Trial 1 finished with value: 0.037691401648998826 and parameters: {'dropout': 0.3148451788551006, 'learning_rate': 3.3284573907694685e-05, 'weight_decay': 0.1813842407190132}. Best is trial 1 with value: 0.037691401648998826.
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.489200,0.458164,0.034126,0.041931,0.037628,0.762888


[I 2025-02-06 23:57:14,248] Trial 2 finished with value: 0.03762827822120867 and parameters: {'dropout': 0.3027022659633645, 'learning_rate': 2.9222558191565536e-05, 'weight_decay': 0.7519791605815216}. Best is trial 1 with value: 0.037691401648998826.


In [6]:
best_trial

BestRun(run_id='1', objective=0.037691401648998826, hyperparameters={'dropout': 0.3148451788551006, 'learning_rate': 3.3284573907694685e-05, 'weight_decay': 0.1813842407190132}, run_summary=None)